<a href="https://colab.research.google.com/github/Amisha1019/Emotion-Detection-Internship-/blob/main/Task_4_emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
nexdatafrank_millions_of_foreign_face_data_single_image_path = kagglehub.dataset_download('nexdatafrank/millions-of-foreign-face-data-single-image')

print('Data source import complete.')


100%|██████████| 26.3M/26.3M [00:00<00:00, 51.9MB/s]

Extracting files...


Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install streamlit opencv-python-headless deepface scikit-learn numpy pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 67.1 MB/s eta 0:00:00


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501


/bin/bash: line 1: streamlit: command not found
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) 

In [4]:
!pip install streamlit


In [ ]:
# Nationality Detection + Emotion + Age + Dress Color
# Streamlit single-file app that:
# - Lets user upload an image (preview shown)
# - Detects face, predicts nationality (using a trained PyTorch model if available)
# - Predicts emotion and age using DeepFace (fallback on simple models if DeepFace unavailable)
# - Predicts dominant dress color by sampling below the face bounding box
# - Conditional outputs:
# * If nationality == 'Indian' -> show nationality, age, emotion, dress color
# * If nationality == 'United States' or 'USA' -> show age, emotion
# * If nationality is an African nationality (user-definable list) -> show emotion and dress color
# * Else -> show nationality and emotion
#
# Notes for the user:
# - Provide a trained nationality classifier at './nationality_model.pth' or put a weights file in
# the Kaggle input path (for example: '/kaggle/input/nationality_model/nationality_model.pth').
# - The dataset path suggested by you: '/kaggle/input/millions-of-foreign-face-data-single-image'
# This script assumes the dataset can be used as an ImageFolder or you can provide a CSV mapping.
# - This file contains both inference (GUI) and a training helper function. Training on large
# datasets should be done on a GPU notebook (Kaggle/Colab) and may take many hours.

In [5]:
!pip install streamlit

In [6]:
import os
import io
import sys
import time
from typing import Tuple, Optional, List

import numpy as np
from PIL import Image, ImageOps

import streamlit as st

In [7]:
# try optional packages; if missing, the app will show instructions
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import DataLoader
    import torchvision
    from torchvision import transforms, models
except Exception:
    torch = None

In [8]:
# DeepFace is optional but recommended for age/emotion convenience
try:
    from deepface import DeepFace
except Exception:
    DeepFace = None

import cv2
from sklearn.cluster import KMeans


25-11-19 13:41:37 - Directory /root/.deepface has been created
25-11-19 13:41:37 - Directory /root/.deepface/weights has been created


In [9]:
def pil_to_cv2(img_pil: Image.Image) -> np.ndarray:
    cv_img = np.array(img_pil.convert('RGB'))
    # Convert RGB to BGR
    return cv_img[:, :, ::-1].copy()


def cv2_to_pil(img_cv: np.ndarray) -> Image.Image:
    # BGR to RGB
    return Image.fromarray(img_cv[:, :, ::-1])

In [10]:
def get_face_detector(net=None):
    # net is a cv2.dnn readNet if provided; otherwise try to use cv2.CascadeClassifier as fallback
    if net is not None:
        return net
    # fallback to Haar cascade available in OpenCV package
    haar_xml = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    if os.path.exists(haar_xml):
        cascade = cv2.CascadeClassifier(haar_xml)
        return cascade
    return None



In [11]:
def detect_faces(image_bgr: np.ndarray, detector=None, conf_threshold=0.5) -> List[Tuple[int,int,int,int]]:
    """Return list of bounding boxes (x, y, w, h) in image coordinates."""
    h, w = image_bgr.shape[:2]
    boxes = []
    if detector is None:
        detector = get_face_detector()

    # If detector is a CascadeClassifier
    if isinstance(detector, cv2.CascadeClassifier):
        gray = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)
        rects = detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30,30))
        for (x,y,ww,hh) in rects:
            boxes.append((x,y,ww,hh))
        return boxes
         # else: try DNN detection (user can supply a dnn net)
    try:
        blob = cv2.dnn.blobFromImage(image_bgr, 1.0, (300,300),(104.0,177.0,123.0))
        detector.setInput(blob)
        detections = detector.forward()
        for i in range(0, detections.shape[2]):
            confidence = detections[0,0,i,2]
            if confidence > conf_threshold:
                box = detections[0,0,i,3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                startX = max(0, startX); startY = max(0, startY)
                endX = min(w-1, endX); endY = min(h-1, endY)
                boxes.append((startX, startY, endX-startX, endY-startY))
        return boxes
    except Exception:
        return boxes

In [12]:
def crop_region(image_bgr: np.ndarray, box: Tuple[int,int,int,int], pad: float=0.15) -> np.ndarray:
    x,y,w,h = box
    H,W = image_bgr.shape[:2]
    pad_x = int(w*pad)
    pad_y = int(h*pad)
    x0 = max(0, x-pad_x); y0 = max(0, y-pad_y)
    x1 = min(W, x+w+pad_x); y1 = min(H, y+h+pad_y)
    return image_bgr[y0:y1, x0:x1]


In [13]:
def get_dominant_color(image_bgr: np.ndarray, k=3) -> Tuple[int,int,int]:
    """Return dominant color as RGB tuple sampled from BGR image."""
    # Convert to RGB
    img = image_bgr[:, :, ::-1]
    img = img.reshape((-1, 3)).astype(np.float32)
    # sample to speed up
    if img.shape[0] > 2000:
        idx = np.random.choice(img.shape[0], 2000, replace=False)
        sample = img[idx]
    else:
        sample = img
    kmeans = KMeans(n_clusters=k, random_state=42).fit(sample)
    counts = np.bincount(kmeans.labels_)
    center = kmeans.cluster_centers_[np.argmax(counts)].astype(int)
    return tuple(int(c) for c in center)


In [14]:
def rgb_to_name(rgb: Tuple[int,int,int]) -> str:
    # Very simple mapping using hue ranges: convert to HSV and map to color names.
    import colorsys
    r,g,b = rgb
    h,s,v = colorsys.rgb_to_hsv(r/255.0,g/255.0,b/255.0)
    hue = h*360
    if v < 0.2:
        return 'black (very dark)'
    if s < 0.2:
        if v > 0.9:
            return 'white'
        else:
            return 'gray'
    if hue < 15 or hue >= 345:
        return 'red'
    if hue < 45:
        return 'orange'
    if hue < 70:
        return 'yellow'
    if hue < 165:
        return 'green'
    if hue < 255:
        return 'blue'
    if hue < 285:
        return 'purple'
    return 'pink'

In [15]:
class SimpleNationalityClassifier(nn.Module):
    def __init__(self, num_classes:int):
        super().__init__()
        self.backbone = models.resnet18(pretrained=True)
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(in_features, num_classes)

    def forward(self,x):
        return self.backbone(x)



In [16]:
def load_nationality_model(model_path: str, device: torch.device, num_classes:int=10):
    if torch is None:
        return None
    model = SimpleNationalityClassifier(num_classes)
    if os.path.exists(model_path):
        try:
            state = torch.load(model_path, map_location=device)
            # support both state_dict and raw model
            if isinstance(state, dict) and 'state_dict' in state:
                model.load_state_dict(state['state_dict'])
            elif isinstance(state, dict) and set(state.keys()).issuperset({'fc.weight','fc.bias'}) or True:
                # assume raw state_dict
                try:
                    model.load_state_dict(state)
                except Exception:
                    # partial load: ignore
                    pass
            else:
                # if object saved
                model = state
        except Exception as e:
            st.warning(f"Could not load model: {e}")
    model.to(device)
    model.eval()
    return model

In [17]:
def train_nationality_classifier(dataset_root: str, output_path: str, epochs=5, batch_size=32, lr=1e-4, num_workers=4):
    if torch is None:
        raise RuntimeError('PyTorch not available in this environment')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # assume dataset_root organized in subfolders per nationality (ImageFolder)
    transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    dataset = torchvision.datasets.ImageFolder(dataset_root, transform=transform)
    num_classes = len(dataset.classes)
    st.write(f'Training with {len(dataset)} images across {num_classes} classes')
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    model = SimpleNationalityClassifier(num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        running = 0.0
        for i, (inputs, labels) in enumerate(loader):
            inputs = inputs.to(device); labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running += loss.item()
            if i % 50 == 0:
                st.write(f'Epoch {epoch+1}/{epochs} step {i} loss {running/(i+1):.4f}')
        # save checkpoint
        torch.save({'state_dict': model.state_dict(), 'classes': dataset.classes}, output_path)
        st.write(f'Saved checkpoint to {output_path}')
    return output_path

In [18]:
def predict_nationality_from_image(model, device, pil_img: Image.Image, class_names: List[str]) -> Tuple[str,float]:
    if model is None or torch is None:
        return 'Unknown', 0.0
    transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    x = transform(pil_img).unsqueeze(0).to(device)
    with torch.no_grad():
        out = model(x)
        probs = torch.nn.functional.softmax(out, dim=1).cpu().numpy()[0]
        idx = int(np.argmax(probs))
        return class_names[idx], float(probs[idx])

st.set_page_config(page_title='Nationality + Emotion Detector', layout='wide')
st.title('Nationality Detection Model — GUI')
st.markdown('/kaggle/input/millions-of-foreign-face-data-single-image/00000007_Brazil_female_young.jpg')



2025-11-19 13:42:11.191 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:11.194 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:11.494 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-11-19 13:42:11.495 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:11.497 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:11.500 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:11.502 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

DeltaGenerator()

In [19]:
col1, col2 = st.columns([1,1])

with col1:
    uploaded = st.file_uploader('/kaggle/input/millions-of-foreign-face-data-single-image/00000047_American_male_young.jpg', type=['jpg','jpeg','png'])
    dataset_path = st.text_input('Optional: dataset root (for training) or folder with classes', value='/kaggle/input/millions-of-foreign-face-data-single-image')
    model_path = st.text_input('Optional: nationality model path (weights)', value='./nationality_model.pth')
    run_train = st.button('Train nationality classifier (use with caution)')

with col2:
    st.write('Model / Environment status:')
    st.write(f"PyTorch available: {'Yes' if torch is not None else 'No'}")
    st.write(f"DeepFace available: {'Yes' if DeepFace is not None else 'No'}")
    st.write('Face detector: Haar cascade fallback used')


2025-11-19 13:42:16.674 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:16.675 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:16.677 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:16.679 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:16.680 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:16.682 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:16.683 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:16.684 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [20]:
if run_train:
    if not os.path.exists(dataset_path):
        st.error('Dataset path not found. Please ensure the dataset is mounted (e.g., in Kaggle input) and the path is correct.')
    else:
        output = st.text_input('Output checkpoint path', value='./nationality_model.pth')
        epochs = st.number_input('Epochs', value=3, min_value=1, max_value=100)
        batch_size = st.number_input('Batch size', value=32, min_value=8, max_value=256)
        st.write('Starting training... open logs below')
        try:
            train_nationality_classifier(dataset_path, output, epochs=int(epochs), batch_size=int(batch_size))
            st.success('Training finished — checkpoint saved')
        except Exception as e:
            st.error(f'Training failed: {e}')

# Load nationality model if possible
device = torch.device('cuda' if (torch is not None and torch.cuda.is_available()) else 'cpu')
nationality_model = None
class_names = []

In [22]:
if torch is not None and os.path.exists(model_path):
    # attempt to load saved checkpoint and classes
    try:
        ckpt = torch.load(model_path, map_location=device)
        if isinstance(ckpt, dict) and 'state_dict' in ckpt and 'classes' in ckpt:
            class_names = ckpt['classes']
            nationality_model = SimpleNationalityClassifier(len(class_names))
            nationality_model.load_state_dict(ckpt['state_dict'])
            nationality_model.to(device); nationality_model.eval()
        elif isinstance(ckpt, dict) and 'classes' in ckpt and 'state_dict' not in ckpt:
            # maybe a straight state_dict with classes appended
            class_names = ckpt.get('classes', [])
            try:
                nationality_model = SimpleNationalityClassifier(len(class_names))
                nationality_model.load_state_dict(ckpt['state_dict'])
                nationality_model.to(device); nationality_model.eval()
            except Exception:
                nationality_model = None
        else:
            # try partial heuristics
            nationality_model = load_nationality_model(model_path, device, num_classes=10)
    except Exception as e:
        st.warning(f'Could not load nationality model: {e}')

american_list_str = st.text_input('Comma-separated list of American nationalities to treat specially', value='Brazil,Spain,American,Thailand')
american_list = [x.strip().lower() for x in american_list_str.split(',') if x.strip()]


2025-11-19 13:42:40.824 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:40.825 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:40.829 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:40.830 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:40.833 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:40.838 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:40.840 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [23]:
if uploaded is not None:
    image = Image.open(uploaded).convert('RGB')
    st.subheader('Input preview')
    st.image(image, use_column_width=True)

    # Convert to cv2 BGR
    img_bgr = pil_to_cv2(image)
    faces = detect_faces(img_bgr)
    if len(faces) == 0:
        st.warning('No faces detected. Try a different image or ensure the face is visible.')
    else:
        st.success(f'Detected {len(faces)} face(s) — showing results for the first face')
        box = faces[0]
        face_crop = crop_region(img_bgr, box, pad=0.25)
        st.image(cv2_to_pil(face_crop), caption='Detected face crop')

        # Predict nationality (using model if available), else fallback to Unknown
        nationality = 'Unknown'
        nat_conf = 0.0
        if nationality_model is not None and class_names:
            face_pil = cv2_to_pil(face_crop)
            nationality, nat_conf = predict_nationality_from_image(nationality_model, device, face_pil, class_names)
        else:
            st.info('Nationality model not found — you can train one or provide a model path.')

        # Predict emotion & age with DeepFace if available
        emotion, age, deepface_raw = predict_emotion_age_deepface(cv2_to_pil(face_crop))
        if DeepFace is None:
            st.info('DeepFace not available. Age and emotion predictions require installing deepface. Falling back to heuristics (not implemented).')
            # Clothing/dress color: crop a region below the face box (approximate torso)
        x,y,w,h = box
        H,W = img_bgr.shape[:2]
        torso_y0 = y + int(0.9*h)
        torso_y1 = min(H, y + int(3.5*h))
        torso_x0 = max(0, x - int(0.5*w))
        torso_x1 = min(W, x + int(1.5*w))
        if torso_y0 < torso_y1 and torso_x0 < torso_x1:
            torso = img_bgr[torso_y0:torso_y1, torso_x0:torso_x1]
            if torso.size == 0:
                dress_color_name = 'Unknown'
                dress_rgb = (0,0,0)
            else:
                dress_rgb = get_dominant_color(torso, k=3)
                dress_color_name = rgb_to_name(dress_rgb)
            st.image(cv2_to_pil(torso), caption='Estimated torso region (for dress color)')
        else:
            dress_color_name = 'Unknown'
            dress_rgb = (0,0,0)

        # Decide which outputs to display depending on nationality
        nat_lower = nationality.lower() if isinstance(nationality, str) else ''
        st.subheader('Predictions')
        col_a, col_b = st.columns(2)
        with col_a:
            st.write('**Nationality**')
            st.write(f'{nationality} (confidence {nat_conf:.2f})')
        with col_b:
            st.write('**Emotion**')
            st.write(f'{emotion if emotion is not None else "Unknown"}')

        # conditional
        if 'indian' in nat_lower or nationality.strip().lower() == 'india':
            st.write('**Additional outputs for Indian nationality**')
            st.write(f'Age: {age if age is not None else "Unknown (DeepFace not available)"}')
            st.write(f'Dress color: {dress_color_name} RGB{dress_rgb}')
        elif nat_lower in ['usa', 'united states', 'united states of america', 'us'] or 'united states' in nat_lower:
            st.write('**US individual: show age & emotion**')
            st.write(f'Age: {age if age is not None else "Unknown (DeepFace not available)"}')
        elif nat_lower in african_list:
            st.write('**Spain individual selected: show emotion & dress color**')
            st.write(f'Dress color: {dress_color_name} RGB{dress_rgb}')
        else:
            st.write('**Default output (other nationalities): nationality & emotion**')

        # Show raw DeepFace output if user wants
        if deepface_raw:
            with st.expander('Show DeepFace raw output'):
                st.write(deepface_raw)

        # Allow download of results as CSV
        import pandas as pd
        results = {
            'filename': uploaded.name,
            'nationality': nationality,
            'nationality_conf': nat_conf,
            'emotion': emotion,
            'age': age,
            'dress_color_name': dress_color_name,
            'dress_color_rgb': str(dress_rgb)
        }
        df = pd.DataFrame([results])
        csv = df.to_csv(index=False).encode('utf-8')
        st.download_button('Download results CSV', data=csv, file_name='prediction_results.csv', mime='text/csv')

st.markdown('---')
st.subheader('Notes & next steps')
st.markdown('''
- This app is a template. For production use:
  - Train a robust nationality classifier on the provided dataset and place the checkpoint path in the model input.
  - Use a stronger face detector (MTCNN or RetinaFace) for better detection accuracy.
  - Consider privacy and ethical implications: predicting nationality from appearance can be inaccurate and harmful. Use responsibly and with informed consent.
- To install dependencies: `pip install streamlit torch torchvision opencv-python-headless deepface scikit-learn`
''')


2025-11-19 13:42:45.196 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:45.198 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:45.198 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:45.199 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:45.200 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:45.201 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:45.202 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 13:42:45.203 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()